In [ ]:
#!pip install keras --upgrade
#!pip uninstall tensorflow
# !pip install tensorflow

In [125]:
import os
# !pip install tensorflow lightfm pandas
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import datetime
import lightfm
from sklearn.utils import class_weight
import numpy as np
import pandas as pd
import tensorflow as tf
from lightfm import LightFM
# from lightfm.datasets import fetch_movielens
from scipy import sparse
import pickle
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import Adam
import time
from __future__ import print_function  # for Python2
import sys
from typing import List


import gc
# hide
print(f"Tensorflow version: {tf.__version__}")
print(f"LightFM version: {lightfm.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

import tensorflow.keras as keras
from tensorflow.keras.layers import (
    Concatenate,
    Dense,
    Embedding,
    Flatten,
    Input,
    Multiply,
    BatchNormalization,
    Dropout,
    Dot,
    Merge
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print ("gpus:", gpus)

### Configurations

# Configurations
TOP_K = 15

products = pd.read_csv('products.csv')

# Define multi-gpu strategy 
BATCHSIZE= 512
mirrored_strategy = tf.distribute.MirroredStrategy()
print (mirrored_strategy.num_replicas_in_sync, 'GPUs in Use')
# Update batch size value
BATCHSIZE = BATCHSIZE * mirrored_strategy.num_replicas_in_sync

sys_details = tf.sysconfig.get_build_info()
cuda_version = sys_details["cuda_version"]
print(cuda_version)
cudnn_version = sys_details["cudnn_version"]  
print(cudnn_version)

Tensorflow version: 2.6.0
LightFM version: 1.16
Pandas version: 1.4.1
Numpy version: 1.19.5


ImportError: cannot import name 'Merge' from 'tensorflow.keras.layers' (/opt/conda/lib/python3.8/site-packages/keras/api/_v2/keras/layers/__init__.py)

### Helper Functions

### Load Train Data

In [ ]:
# df_train = pd.read_csv("DLS_Project/interaction_prior_20k_rank250_100recs_with_hardnegatives_2neighbors.csv")

# class_weights = class_weight.compute_class_weight(
#                'balanced',
#                 np.unique(df_train.interaction.tolist()), 
#                 df_train.interaction.tolist()) 

# train_user_n, train_product_n = df_train.user_id.nunique(), 49688
# print (df_train.shape)
# print (class_weights)
# # print (df_train.interaction.value_counts(normalize=True))
# # print (train_user_n)
# # df_train.head()

In [ ]:
# df_train.head()

### Reading in Instacart Feature Vec

In [15]:
df_instacart_product_vectors = pd.read_csv("instacart_feature_vec.csv", header=None)

df_instacart_product_vectors.head()

df_instacart_product_vectors = df_instacart_product_vectors.drop(101, axis=1)

df_instacart_product_vectors = df_instacart_product_vectors.sort_values(0)

df_instacart_product_vectors = df_instacart_product_vectors.drop(0, axis=1)

np_instacart_product_vectors = df_instacart_product_vectors.to_numpy()

np_instacart_product_vectors = np.insert(np_instacart_product_vectors, 0, np.zeros(100), 0)

print (np_instacart_product_vectors.shape)

# only if we're using negative sampling
# np_instacart_product_vectors = np.take(np_instacart_product_vectors, w2v_indices, axis=0)

# df_train.product_id.nunique()
# del df_instacart_product_vectors
# gc.collect()
# df_instacart_product_vectors = pd.DataFrame()

(49688, 100)


In [ ]:
# products['features'] = pd.Series(np_instacart_product_vectors.tolist())

# products

In [ ]:
## Reading in Pre-Trained MF Embeddings

# file_name = 'user_embedding_MF_ALS.pkl'
# open_file = open(file_name, "rb")
# user_embeddings = pickle.load(open_file)
# open_file.close()

# file_name = 'product_embedding_MF_ALS.pkl'
# open_file = open(file_name, "rb")
# product_embeddings = pickle.load(open_file)
# open_file.close()

In [ ]:
# print (type(user_embeddings), user_embeddings.shape)
# print (type(product_embeddings), product_embeddings.shape)

# To Do

- Figure out why there's 49687 unique products in training set while there are actually 49688 unique products in total
- Try calibration layer / regression on top of negative sampling NeuFM
- Run just MF save embeddings and use embedding look up to freeze weights, while training NN side of network with negative sampling
- Once this is done, we move to optimizing the NN with various hyperparameters
- figure out how to use tensorboard
- trying to get to 0.21, 0.22 MAP value


In [ ]:
# np_instacart_product_vectors.shape

In [ ]:
# Run if randomized index

# user_to_zerobased_index = dict(zip(df_train.user_id.unique(), range(df_train.user_id.nunique())))
# product_to_zerobased_index = dict(zip(df_train.product_id.unique(), range(df_train.product_id.nunique())))

# zerobased_index_to_product = {value: key for key, value in product_to_zerobased_index.items()}

# zerobased_index_to_user = {value: key for key, value in user_to_zerobased_index.items()}

# df_train.user_id = df_train.user_id.apply(lambda x: user_to_zerobased_index[x])
# df_train.product_id = df_train.product_id.apply(lambda x: product_to_zerobased_index[x])

In [45]:
# Function to sort the list by second item of tuple
def Sort_Tuple(tup): 
  
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    return(sorted(tup, key = lambda x: x[1], reverse=True))

from pyspark.mllib.evaluation import RankingMetrics
from pyspark.sql.functions import broadcast
from pyspark import StorageLevel
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession


def make_tf_dataset_train(
    df: pd.DataFrame,
    targets: List[str],
    val_split: float = 0.2,
    batch_size: int = BATCHSIZE,
    seed=28,
):
    """Make TensorFlow dataset from Pandas DataFrame.
    :param df: input DataFrame - only contains features and target(s)
    :param targets: list of columns names corresponding to targets
    :param val_split: fraction of the data that should be used for validation
    :param batch_size: batch size for training
    :param seed: random seed for shuffling data - `None` won't shuffle the data"""

    n_val = round(df.shape[0] * val_split)
    if seed:
        # shuffle all the rows
        x = df.sample(frac=1, random_state=seed).to_dict("series")
    else:
        x = df.to_dict("series")
    y = dict()
    for t in targets:
        y[t] = x.pop(t)
    interaction_terms = y[t].values
    sample_weight = np.ones(shape=(len(interaction_terms),))
    #sample_weight[interaction_terms == 0] = 2.037969065266319
    #sample_weight[interaction_terms == 1] = 0.6625520341377668
    sample_weight[interaction_terms == 0] = class_weights[0]
    sample_weight[interaction_terms == 1] = class_weights[1]
#     print (sample_weight)
#         y[t] = tf.keras.utils.to_categorical(x.pop(t))
#     print (y.values())
#     print (x.values())
    ds = tf.data.Dataset.from_tensor_slices((x, y, sample_weight))
    
    #ds = ds.cache()
    
    ds_val = ds.take(n_val).batch(batch_size)
    #ds_val = ds_val.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    ds_train = ds.skip(n_val).batch(batch_size)
    #ds_train = ds_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds_train, ds_val

def make_tf_dataset_val(
    df: pd.DataFrame,
    targets: List[str],
    val_split: float = 0.1,
    batch_size: int = BATCHSIZE,
    seed=28,
):
    """Make TensorFlow dataset from Pandas DataFrame.
    :param df: input DataFrame - only contains features and target(s)
    :param targets: list of columns names corresponding to targets
    :param val_split: fraction of the data that should be used for validation
    :param batch_size: batch size for training
    :param seed: random seed for shuffling data - `None` won't shuffle the data"""

    n_val = round(df.shape[0] * val_split)
    if seed:
        # shuffle all the rows
        x = df.sample(frac=1, random_state=seed).to_dict("series")
    else:
        x = df.to_dict("series")
    y = dict()
    for t in targets:
        y[t] = x.pop(t)

#         y[t] = tf.keras.utils.to_categorical(x.pop(t))
#     print (y.values())
#     print (x.values())
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    
    #ds = ds.cache()
    
    ds_val = ds.take(n_val).batch(batch_size)
    #ds_val = ds_val.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    ds_train = ds.skip(n_val).batch(batch_size)
    #ds_train = ds_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds_train, ds_val

def grab_memory_usage(local_var):
    vars_list = []
    sizes = []

    local_vars = list(local_var.items())
    for var, obj in local_vars:
        vars_list.append(var)
        sizes.append(sys.getsizeof(obj))
        #print(var, sys.getsizeof(obj))

    df_var_sizes = pd.DataFrame([vars_list, sizes]).T
    df_var_sizes.columns = ['variable', 'sizes']
    df_var_sizes = df_var_sizes.sort_values("sizes", ascending=False).reset_index(drop=True)
    df_var_sizes.sizes = df_var_sizes.sizes/1000000000.00
    return df_var_sizes

# grab_memory_usage(locals())

class MemoryPrintingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        #gpu_dict = tf.config.experimental.get_memory_info('GPU:0')
        gpu_usage = tf.config.experimental.get_memory_usage('GPU:0')
        tf.print('\n GPU memory details [current: {} gb]'.format(
          float(gpu_usage) / (1024 ** 3)))
#         tf.print('\n GPU memory details [current: {} gb, peak: {} gb]'.format(
#           float(gpu_dict['current']) / (1024 ** 3), 
#           float(gpu_dict['peak']) / (1024 ** 3)))

# tf.config.experimental.get_memory_usage('GPU:0') / (1024**3)

# Saving Model

def save_model_artifacts(train_hist, ncf_model, train_filename, model_architecture, dropout_input,dropout_hidden_layer, dropout_on_multiplication_boolean, dropout_on_multiplication,
                        latent_dim_mf, latent_dim_mlp, reg_mf, reg_mlp, dense_layers, reg_layers, N_EPOCHS, BATCHSIZE):

    pd.DataFrame.from_dict(train_hist.history).to_csv(str(train_filename) + "_" + str(model_architecture) + "_" + str(dropout_input) + "_" + str(dropout_hidden_layer) +
                                                     "_" + str(dropout_on_multiplication_boolean) + "_" + str(dropout_on_multiplication) + "_" + str(latent_dim_mlp) + "_" + str(latent_dim_mf) +
                                                     "_" + str(reg_mf) + "_" + str(reg_mlp) + "_" + str(dense_layers) + "_" + str(reg_layers) + "_" + str(N_EPOCHS) + "_" + str(BATCHSIZE) + "_" + str(batch_norm_boolean) + "_" + str(dropout_on_multiplication_boolean) 
                                                      + "_" + str(dropout_on_multiplication) + ".csv")
    
#     ncf_model.save("~/work/DLS_Project/model_" + str(train_filename) + "_" + str(model_architecture) + "_" + str(dropout_input) + "_" + str(dropout_hidden_layer) +
#                                                      "_" + str(dropout_on_multiplication_boolean) + "_" + str(dropout_on_multiplication) + "_" + str(latent_dim_mlp) + "_" + str(latent_dim_mf) +
#                                                      "_" + str(reg_mf) + "_" + str(reg_mlp) + "_" + str(dense_layers) + "_" + str(reg_layers))   


# Loading Validation and Making Predictions

def load_val_make_preds(ncf_model, val_filename, gt_filename):

    ### Load Validation Set

    df_val = pd.read_csv(val_filename)
    df_truth = pd.read_pickle(gt_filename)
    ds_test, _ = make_tf_dataset_val(df_val, ['interaction'], val_split=0, seed=None)
    ncf_predictions = ncf_model.predict(ds_test, batch_size=512, max_queue_size=500, verbose=0)
    df_val["ncf_predictions"] = ncf_predictions
    df_val["pred"] = (df_val.ncf_predictions >= 0.5) * 1.0
    df_val["correct"] = (df_val.interaction == df_val.pred) * 1.0
    df_val.drop_duplicates(inplace=True)
    return df_val, df_truth

def evaluate_and_save_results(df_val, df_truth):
#     std = df_val.ncf_predictions.std()
#     if std < 0.01:
#         raise ValueError("Model predictions have standard deviation of less than 1e-2.")

    all_indices_predictions = df_val.user_id.unique()
    all_indices_predictions.sort()

    sorted_df_val = df_val[['user_id', 'product_id', 'ncf_predictions']].sort_values(by=['user_id', 'ncf_predictions'], ascending=[True, False])
    gb_sorted_df_val = sorted_df_val.groupby('user_id')
    gb_df_truth = df_truth.groupby('user_id')

    predictionAndLabelsAll = []
    counter = 0
    for i in all_indices_predictions:
        if counter % 5000 == 0:
            print (counter)
        counter+=1 
        preds = gb_sorted_df_val.get_group(i).product_id.tolist()
        truth = gb_df_truth.get_group(i).product_id.tolist()
        predictionAndLabelsAll.append((preds, truth))

    predictionAndLabels_All = sc.parallelize(predictionAndLabelsAll)
    metrics_all = RankingMetrics(predictionAndLabels_All)
    
    return (metrics_all.meanAveragePrecisionAt(15), metrics_all.ndcgAt(15))

### Define Model

In [17]:
with mirrored_strategy.scope():
    
    def lr_scheduler(epoch, lr):
        if epoch > 1:
            lr = 0.0005
        if epoch > 3:
            lr = 0.0001
            return lr
        if epoch > 5:
            lr = 0.00005
        return lr
    
    def generate_embedding(input_dim, output_dim, name, reg_mf):
        return Embedding(
                input_dim=input_dim,
                output_dim=output_dim,
                name=name,
                embeddings_initializer="RandomNormal",
                embeddings_regularizer=l2(reg_mf),
                input_length=1,
            )
    
    def create_ncf(
        number_of_users: int,
        number_of_items: int,
        latent_dim_mf: int = 32,
        latent_dim_mlp: int = 64,
        reg_mf: int = 0,
        reg_mlp: int = 0.001,
        dense_layers: List[int] = [1024, 512, 256],
        reg_layers: List[int] = [0.001, 0.001, 0.001],
        activation_dense: str = "relu",
    ) -> keras.Model:

        # input layer
        user = Input(shape=(), dtype="int32", name="user_id")
        item = Input(shape=(), dtype="int32", name="product_id")


        embedding_dim = 100
        num_tokens = n_items

        product_embedding_layer = Embedding(
            num_tokens,
            embedding_dim,
            embeddings_initializer=keras.initializers.Constant(np_instacart_product_vectors),
            #embeddings_initializer=keras.initializers.VarianceScaling(np_instacart_product_vectors),
            trainable=False,
            input_length=1,
        )

    #     product_mf_embedding_dim = 250
    #     num_tokens_products = train_product_n

    #     pretrained_product_embedding_mf = Embedding(
    #         num_tokens_products,
    #         product_mf_embedding_dim,
    #         embeddings_initializer=keras.initializers.Constant(product_embeddings),
    #         #embeddings_initializer=keras.initializers.VarianceScaling(np_instacart_product_vectors),
    #         trainable=False,
    #         input_length=1,
    #     )

    #     user_mf_embedding_dim = 250
    #     num_tokens_users = train_user_n

    #     pretrained_user_embedding_mf = Embedding(
    #         num_tokens_users,
    #         user_mf_embedding_dim,
    #         embeddings_initializer=keras.initializers.Constant(user_embeddings),
    #         #embeddings_initializer=keras.initializers.VarianceScaling(np_instacart_product_vectors),
    #         trainable=False,
    #         input_length=1,
    #     )

        #embedding layers
        
        mf_user_embedding = generate_embedding(number_of_users, latent_dim_mf, "mf_user_embedding", reg_mf)
        mf_item_embedding = generate_embedding(number_of_items, latent_dim_mf, "mf_item_embedding", reg_mf)
        
        mlp_user_embedding = generate_embedding(number_of_users, latent_dim_mlp, "mlp_user_embedding", reg_mlp)
        mlp_item_embedding = generate_embedding(number_of_items, latent_dim_mlp, "mlp_item_embedding", reg_mlp)
        
        mlp_user_embedding_wv = generate_embedding(number_of_users, latent_dim_mlp, "mlp_user_embedding", reg_mlp)
        mlp_item_embedding_wv = generate_embedding(number_of_items, latent_dim_mlp, "mlp_item_embedding", reg_mlp)

        # word2vec embeddings
        product_vector = Flatten()(product_embedding_layer(item))

        # Pretrained MF embeddings

    #     pretrained_mf_item_latent = Flatten()(pretrained_product_embedding_mf(item))
    #     pretrained_mf_user_latent = Flatten()(pretrained_user_embedding_mf(user))
    #     mf_cat_latent = Multiply()([pretrained_mf_user_latent, pretrained_mf_item_latent])

        # MF vector

        mf_user_latent = Flatten()(mf_user_embedding(user))
        mf_item_latent = Flatten()(mf_item_embedding(item))
        mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])

        # MLP vector 1
        mlp_user_latent = Flatten()(mlp_user_embedding(user))
        mlp_item_latent = Flatten()(mlp_item_embedding(item))
        
        # MLP vector 2
        mlp_user_latent_wv = Flatten()(mlp_user_embedding_wv(user))
        mlp_item_latent_wv = Flatten()(mlp_item_embedding_wv(item))


        # MLP vector 1 Concatenation
        
        mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])

        # MLP vector 2 Concatenation

        mlp_cat_latent_wv = Concatenate()([mlp_user_latent_wv, product_vector])
        
#         mlp_user_w2v = Concatenate()([mlp_user_latent, product_vector])
#         mlp_item_w2v = Concatenate()([mlp_item_latent, product_vector])
#         mlp_3pronged = Concatenate()([mlp_user_latent, product_vector, mlp_item_latent])
#         mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])
#         mlp_cat_latent = Concatenate()([mlp_user_w2v, mlp_item_w2v])
        
        mlp_vector = mlp_cat_latent
        
    
        # Start of NN Architecture
    
        mlp_vector = Dropout(0.5)(mlp_vector)
        relu_layer = tf.keras.layers.ReLU()
        # build dense layers for model
        for i in range(len(dense_layers)):
            layer = Dense(
                dense_layers[i],
                activity_regularizer=l2(reg_layers[i]),
#                 activation=activation_dense,
                activation='linear',
                name="layer%d" % i,
            )
            mlp_vector = layer(mlp_vector)
            mlp_vector = relu_layer(mlp_vector)
            #mlp_vector = BatchNormalization()(mlp_vector)
            mlp_vector = Dropout(0.2)(mlp_vector)

    #     # build dense layers for model
    #     for i in range(len(dense_layers)):
    #         layer = Dense(
    #             dense_layers[i],
    #             activity_regularizer=l2(reg_layers[i]),
    #             activation=activation_dense,
    #             name="wv_layer%d" % i,
    #         )
    #         mlp_vector_wv = layer(mlp_vector_wv)  

        #predict_layer = Concatenate()([mf_cat_latent, mlp_vector])
        predict_layer = mlp_vector
        #predict_layer = Concatenate()([mf_cat_latent, mlp_vector, product_vector])

        result = Dense(
            1, activation="sigmoid", kernel_initializer="lecun_uniform", name="interaction"
        )
    
        #predict_layer_post_dense = intermediate_result(predict_layer)
        output = result(predict_layer)
        #output = Flatten()(result(predict_layer))
        #output = result(mf_cat_latent)

        model = Model(
            inputs=[user, item],
            outputs=[output],
        )

        return model

In [18]:
# import tf_traindataset


# ds_train, ds_val = make_tf_dataset_train(df_train, ['interaction'])
# del df_train
# gc.collect()
# df_train = pd.DataFrame()

# n_users, n_items = train_user_n, train_product_n
# with mirrored_strategy.scope():

#     print ("batchsize:", BATCHSIZE)
#     # policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
#     # tf.keras.mixed_precision.experimental.set_policy(policy) 

#     ncf_model = create_ncf(n_users, n_items)

#     ncf_model.compile(
#         optimizer=Adam(learning_rate=0.001),
# #         loss="binary_crossentropy",
#         loss='binary_crossentropy',
#         #loss="hinge",
#         metrics=[
#             tf.keras.metrics.TruePositives(name="tp"),
#             tf.keras.metrics.FalsePositives(name="fp"),
#             tf.keras.metrics.TrueNegatives(name="tn"),
#             tf.keras.metrics.FalseNegatives(name="fn"),
#             tf.keras.metrics.BinaryAccuracy(name="accuracy"),
#             tf.keras.metrics.Precision(name="precision"),
#             tf.keras.metrics.Recall(name="recall"),
#             tf.keras.metrics.AUC(name="auc")
#         ],
#         weighted_metrics=[tf.keras.metrics.BinaryAccuracy(name="weighted_accuracy"),
#             tf.keras.metrics.Precision(name="weighted_precision"),
#             tf.keras.metrics.Recall(name="weighted_recall"),
#             tf.keras.metrics.AUC(name="weighted_auc")]
#     )
#     ncf_model._name = "neural_collaborative_filtering"
#     #print (ncf_model.summary())
#     logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#     tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch=(1,2))
#     early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#         monitor="val_precision", patience=2, restore_best_weights = True,
#     )
    
#     lr_scheduler_callback = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

#     train_hist = ncf_model.fit(
#         ds_train,
#         validation_data=ds_val,
#         epochs=N_EPOCHS,
#         #callbacks=[tensorboard_callback, early_stopping_callback],
#         #callbacks=[tensorboard_callback, lr_scheduler_callback, MemoryPrintingCallback()],
#         callbacks=[tensorboard_callback],
#         verbose=1,
#         shuffle=True,
#         workers=12,
#         use_multiprocessing=True,
#         max_queue_size=500,

#     )
    

In [ ]:
0.9 --> 1, 0.09 --> 0.1

In [147]:
with mirrored_strategy.scope():
    
    def lr_scheduler(epoch, lr):
        if epoch > 1:
            lr = 0.0001
        if epoch > 3:
            lr = 0.00009
        if epoch > 7:
            lr = 0.0005
        if epoch > 9:
            lr = 0.0003
        if epoch > 12:
            lr = 0.0001
        if epoch > 30:
            lr = 0.00009
        if epoch > 35:
            lr = 0.00007
        return lr
    
    def generate_embedding(input_dim, output_dim, name, reg_mf):
        return Embedding(
                input_dim=input_dim,
                output_dim=output_dim,
                name=name,
                embeddings_initializer="RandomNormal",
                embeddings_regularizer=l2(reg_mf),
                input_length=1,
            )
    
    
    def create_ncf(
        architecture,
        dropout_input,
        dropout_hidden_layer,
        dropout_on_multiplication_boolean,
        dropout_on_multiplication,
        batch_norm_boolean,
        number_of_users: int,
        number_of_items: int,
        latent_dim_mf: int = 32,
        latent_dim_mlp: int = 64,
        reg_mf: int = 0,
        reg_mlp: int = 0.001,
        dense_layers: List[int] = [1024, 512, 256],
        reg_layers: List[int] = [0.001, 0.001, 0.001],
        activation_dense: str = "relu"
        
    ) -> keras.Model:

        # input layer
        user = Input(shape=(), dtype="int32", name="user_id")
        item = Input(shape=(), dtype="int32", name="product_id")
        
        embedding_dim = 100
        num_tokens = number_of_items

        product_embedding_layer = Embedding(
            num_tokens,
            embedding_dim,
            embeddings_initializer=keras.initializers.Constant(np_instacart_product_vectors),
            #embeddings_initializer=keras.initializers.VarianceScaling(np_instacart_product_vectors),
            trainable=False,
            input_length=1,
            name='w2v_product_vector'
        )

    #     product_mf_embedding_dim = 250
    #     num_tokens_products = train_product_n

    #     pretrained_product_embedding_mf = Embedding(
    #         num_tokens_products,
    #         product_mf_embedding_dim,
    #         embeddings_initializer=keras.initializers.Constant(product_embeddings),
    #         #embeddings_initializer=keras.initializers.VarianceScaling(np_instacart_product_vectors),
    #         trainable=False,
    #         input_length=1,
    #     )

    #     user_mf_embedding_dim = 250
    #     num_tokens_users = train_user_n

    #     pretrained_user_embedding_mf = Embedding(
    #         num_tokens_users,
    #         user_mf_embedding_dim,
    #         embeddings_initializer=keras.initializers.Constant(user_embeddings),
    #         #embeddings_initializer=keras.initializers.VarianceScaling(np_instacart_product_vectors),
    #         trainable=False,
    #         input_length=1,
    #     )

        #embedding layers
        
        mf_user_embedding = generate_embedding(number_of_users, latent_dim_mf, "mf_user_embedding", reg_mf)
        mf_item_embedding = generate_embedding(number_of_items, latent_dim_mf, "mf_item_embedding", reg_mf)
        
        mlp_user_embedding = generate_embedding(number_of_users, latent_dim_mlp, "mlp_user_embedding", reg_mlp)
        mlp_item_embedding = generate_embedding(number_of_items, latent_dim_mlp, "mlp_item_embedding", reg_mlp)
        
        mlp_user_embedding_wv = generate_embedding(number_of_users, latent_dim_mlp, "mlp_user_embedding_wv", reg_mlp)
        mlp_item_embedding_wv = generate_embedding(number_of_items, latent_dim_mlp, "mlp_item_embedding_wv", reg_mlp)

        # word2vec embeddings
        product_vector = Flatten()(product_embedding_layer(item))

        # Pretrained MF embeddings

    #     pretrained_mf_item_latent = Flatten()(pretrained_product_embedding_mf(item))
    #     pretrained_mf_user_latent = Flatten()(pretrained_user_embedding_mf(user))
    #     mf_cat_latent = Multiply()([pretrained_mf_user_latent, pretrained_mf_item_latent])

        # MF vector

        mf_user_latent = Flatten()(mf_user_embedding(user))
        mf_item_latent = Flatten()(mf_item_embedding(item))
        #mf_cat_latent = Dot(axes=1)([mf_user_latent, mf_item_latent])
        mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])
        if dropout_on_multiplication_boolean:
            mf_cat_latent = Dropout(dropout_on_multiplication)(mf_cat_latent)
        # MLP vector 1
        mlp_user_latent = Flatten()(mlp_user_embedding(user))
        mlp_item_latent = Flatten()(mlp_item_embedding(item))
        
        # MLP vector 2
        mlp_user_latent_wv = Flatten()(mlp_user_embedding_wv(user))
        mlp_item_latent_wv = Flatten()(mlp_item_embedding_wv(item))


        # MLP vector 1 Concatenation
        
        mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])

        # MLP vector 2 Concatenation

        mlp_cat_latent_wv = Concatenate()([mlp_user_latent_wv, product_vector])
        
#         mlp_user_w2v = Concatenate()([mlp_user_latent, product_vector])
#         mlp_item_w2v = Concatenate()([mlp_item_latent, product_vector])
#         mlp_3pronged = Concatenate()([mlp_user_latent, product_vector, mlp_item_latent])
#         mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])
#         mlp_cat_latent = Concatenate()([mlp_user_w2v, mlp_item_w2v])
        
        if (architecture == 'vanilla_NCF') or (architecture == 'mf_only') or (architecture == 'vanilla_NCF_right'):    
            mlp_vector = mlp_cat_latent
        elif (architecture == 'w2v_NCF') or (architecture == 'w2v_NCF_right'):
            mlp_vector = Concatenate()([mlp_cat_latent, mlp_cat_latent_wv])
        

        # Start of NN Architecture
    
        mlp_vector = Dropout(dropout_input)(mlp_vector)
        # build dense layers for model
        for i in range(len(dense_layers)):
            layer = Dense(
                dense_layers[i],
                activity_regularizer=l2(reg_layers[i]),
#                 activation=activation_dense,
                activation='linear',
                name="layer%d" % i,
            )
            mlp_vector = layer(mlp_vector)
            relu_layer = tf.keras.layers.ReLU()
            mlp_vector = relu_layer(mlp_vector)
            mlp_vector = Dropout(dropout_hidden_layer)(mlp_vector)
            if batch_norm_boolean:
                mlp_vector = BatchNormalization()(mlp_vector)

    #     # build dense layers for model
    #     for i in range(len(dense_layers)):
    #         layer = Dense(
    #             dense_layers[i],
    #             activity_regularizer=l2(reg_layers[i]),
    #             activation=activation_dense,
    #             name="wv_layer%d" % i,
    #         )
    #         mlp_vector_wv = layer(mlp_vector_wv)  

        if (architecture == 'mf_only'):
            predict_layer = mf_cat_latent
        elif (architecture == 'vanilla_NCF_right') or (architecture == 'w2v_NCF_right'):
            predict_layer = mlp_vector
        elif (architecture == 'vanilla_NCF') or (architecture == 'w2v_NCF'):
            predict_layer = Concatenate()([mf_cat_latent, mlp_vector])

        result = Dense(
            1, activation="sigmoid", kernel_initializer="lecun_uniform", name="interaction"
        )
    
        #predict_layer_post_dense = intermediate_result(predict_layer)
        output = result(predict_layer)
        #output = Flatten()(result(predict_layer))
        #output = result(mf_cat_latent)

        model = Model(
            inputs=[user, item],
            outputs=[output],
        )

        return model

# Pipeline Start

In [133]:
SparkContext.setSystemProperty('spark.executor.memory', '32g')
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .config("spark.driver.memory", "64g")\
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)
sc.setLogLevel("ERROR")

/opt/conda/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
configurations = ["interaction_prior_20k_rank250_100recs", "interaction_prior_20k_rank250_100recs_with_hardnegatives_2neighbors", "interaction_prior_20k_rank250_100recs_with_hardnegatives_4neighbors",
                 "interaction_prior_20k_rank250_100recs_with_hardnegatives_6neighbors", "interaction_prior_20k_rank250_100recs_with_hardnegatives_8neighbors",
                 "interaction_prior_20k_rank250_100recs_with_hardnegatives_10neighbors"]

In [ ]:
# configurations = [("interaction_prior_20k_rank250_100recs", 'vanilla_NCF_right'), ("interaction_prior_20k_rank250_100recs_with_hardnegatives_4neighbors", 'vanilla_NCF_right'), 
#                   ("interaction_prior_20k_rank250_100recs", 'w2v_NCF_right'), ("interaction_prior_20k_rank250_100recs_with_hardnegatives_4neighbors", 'w2v_NCF_right')]

configurations = [("interaction_prior_20k_rank250_100recs", 'w2v_NCF'), ("interaction_prior_20k_rank250_100recs", 'vanilla_NCF')]

#, ("interaction_prior_20k_rank250_100recs_with_hard_negatives(1)_withpositives_withrns(5)", 'w2v_NCF_right')

In [ ]:
for i in range(len(configurations)):
    #train_filename = "interaction_prior_20k_rank250_100recs_with_hardnegatives_2neighbors"
    train_filename = configurations[i][0]
    architecture = configurations[i][1]
    dropout_input = 0.5
    dropout_hidden_layer = 0.2
    dropout_on_multiplication_boolean = True
    dropout_on_multiplication = 0.3
    latent_dim_mf  = 256
    latent_dim_mlp = 128
    reg_mf = 0
    reg_mlp = 0.001
    dense_layers = [1024, 512, 256]
    reg_layers = [0.001, 0.001, 0.001]
    activation_dense = "relu"
    N_EPOCHS = 10
    BATCHSIZE = 128
    batch_norm_boolean = True

    val_filename = "DLS_Project/interaction_train_20k_rank250_100recs.csv"
    ground_truth_filename = "DLS_Project/cg_interaction_ground_truth_20k_rank250_100recs.pkl"
    results_filename = "DLS_Project/Test5_VanillaNCF_w2vNCF_testing_with_mfdropout.txt"

    BATCHSIZE = BATCHSIZE * mirrored_strategy.num_replicas_in_sync

    df_train = pd.read_csv("DLS_Project/" + train_filename + ".csv")

    class_weights = class_weight.compute_class_weight(
                   'balanced',
                    np.unique(df_train.interaction.tolist()), 
                    df_train.interaction.tolist()) 

    train_user_n, train_product_n = df_train.user_id.nunique(), 49688
    # print (df_train.shape)

    print (train_filename)

    ds_train, ds_val = make_tf_dataset_train(df_train, ['interaction'], batch_size=BATCHSIZE)
    del df_train
    gc.collect()
    df_train = pd.DataFrame()

    with mirrored_strategy.scope():

        print ("batchsize:", BATCHSIZE)
        # policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
        # tf.keras.mixed_precision.experimental.set_policy(policy) 

        ncf_model = create_ncf(architecture, dropout_input, dropout_hidden_layer, dropout_on_multiplication_boolean, dropout_on_multiplication, batch_norm_boolean, train_user_n, train_product_n,
                               latent_dim_mf, latent_dim_mlp, reg_mf, reg_mlp, dense_layers, reg_layers)

        ncf_model.compile(
            optimizer=Adam(learning_rate=0.001),
    #         loss="binary_crossentropy",
            loss='binary_crossentropy',
            #loss="hinge",
            metrics=[
                tf.keras.metrics.TruePositives(name="tp"),
                tf.keras.metrics.FalsePositives(name="fp"),
                tf.keras.metrics.TrueNegatives(name="tn"),
                tf.keras.metrics.FalseNegatives(name="fn"),
                tf.keras.metrics.BinaryAccuracy(name="accuracy"),
                tf.keras.metrics.Precision(name="precision"),
                tf.keras.metrics.Recall(name="recall"),
                tf.keras.metrics.AUC(name="auc")
            ],
            weighted_metrics=[tf.keras.metrics.BinaryAccuracy(name="weighted_accuracy"),
                tf.keras.metrics.Precision(name="weighted_precision"),
                tf.keras.metrics.Recall(name="weighted_recall"),
                tf.keras.metrics.AUC(name="weighted_auc")]
        )
        ncf_model._name = "neural_collaborative_filtering"
        print (ncf_model.summary())
        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch=(1,2))
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_precision", patience=2, restore_best_weights = True,
        )

        lr_scheduler_callback = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

        train_hist = ncf_model.fit(
            ds_train,
            validation_data=ds_val,
            epochs=N_EPOCHS,
            #callbacks=[tensorboard_callback, early_stopping_callback],
            #callbacks=[tensorboard_callback, lr_scheduler_callback, MemoryPrintingCallback()],
            #callbacks=[tensorboard_callback],
            callbacks=[lr_scheduler_callback],
            verbose=1,
            shuffle=True,
            workers=12,
            use_multiprocessing=True,
            max_queue_size=500,
        )

    df_val, df_truth = load_val_make_preds(ncf_model, val_filename, ground_truth_filename)

    map_15, ndcg_15 = evaluate_and_save_results(df_val, df_truth)

    print (map_15, ndcg_15)
    
    
    # Comment if you don't want to write out results

    file1 = open(results_filename,"a")
    L = [str(train_filename) + "_" + str(architecture) + "_" + str(dropout_input) + "_" + str(dropout_hidden_layer) +
                                "_" + str(dropout_on_multiplication_boolean) + "_" + str(dropout_on_multiplication) + "_" + str(latent_dim_mlp) + "_" + str(latent_dim_mf) +
                                "_" + str(reg_mf) + "_" + str(reg_mlp) + "_" + str(dense_layers) + "_" + str(reg_layers) + "_" + str(N_EPOCHS) + "_" + str(BATCHSIZE) + "_" + str(batch_norm_boolean) + "_" + str(dropout_on_multiplication_boolean) + 
                                                      "_" + str(dropout_on_multiplication) + "\n",
        "MAP 15: " + str(map_15) + "\n",
        "NDCG 15: " + str(ndcg_15) + "\n", "\n \n"]
    file1.writelines(L)
    file1.close() #to change file access modes
    
    save_model_artifacts(train_hist, ncf_model, train_filename, architecture, dropout_input, dropout_hidden_layer, dropout_on_multiplication_boolean, dropout_on_multiplication,
                            latent_dim_mf, latent_dim_mlp, reg_mf, reg_mlp, dense_layers, reg_layers, N_EPOCHS, BATCHSIZE)

    del ncf_model 
    tf.keras.backend.clear_session()
    gc.collect()

In [ ]:
(.2078-0.1798)/(0.1798)

In [ ]:
# 0.12299531070321705 0.23477773249508563

# Pipeline End

# Pipeline 2 Start

In [148]:
# configurations = [("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 5),
#                   ("interaction_prior_20k_rank250_100recs_withrns(5)", 'w2v_NCF', 5),
#                   ("interaction_prior_20k_rank250_100recs_with_hard_negatives(1)_withpositives_withrns(5)", 'w2v_NCF', 5),
#                  ("interaction_prior_20k_rank250_100recs", 'w2v_NCF_right', 10),
#                   ("interaction_prior_20k_rank250_100recs_withrns(5)", 'w2v_NCF_right', 10),
#                   ("interaction_prior_20k_rank250_100recs_with_hard_negatives(1)_withpositives_withrns(5)", 'w2v_NCF_right', 10),
#                  ("interaction_prior_20k_rank250_100recs", 'w2v_NCF_right', 15),
#                   ("interaction_prior_20k_rank250_100recs_withrns(5)", 'w2v_NCF_right', 15),
#                   ("interaction_prior_20k_rank250_100recs_with_hard_negatives(1)_withpositives_withrns(5)", 'w2v_NCF_right', 15)]


configurations = [("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 1, 256),
                 ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 2, 256),
                  ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 3, 256),
                 ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 4, 256)]

# configurations = [("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 1, 32),
#                   ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 2, 32),
#                   ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 3, 32),
#                   ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 4, 32),
#                   ("interaction_prior_20k_rank250_100recs", 'w2v_NCF', 5, 32)]

In [149]:
for i in range(len(configurations)):
    #train_filename = "interaction_prior_20k_rank250_100recs_with_hardnegatives_2neighbors"
    train_filename = configurations[i][0]
    architecture = configurations[i][1]
    dropout_input = 0.5
    dropout_hidden_layer = 0.2
    dropout_on_multiplication_boolean = False
    dropout_on_multiplication = 0.7
    latent_dim_mf  = configurations[i][3]
    latent_dim_mlp = 180
    reg_mf = 0
    reg_mlp = 0.001
    dense_layers = [512, 256]
    reg_layers = [0.001, 0.001]
    activation_dense = "relu"
    N_EPOCHS = configurations[i][2]
    BATCHSIZE = 128
    batch_norm_boolean = False

    val_filename = "DLS_Project/interaction_train_20k_rank250_100recs.csv"
    ground_truth_filename = "DLS_Project/cg_interaction_ground_truth_20k_rank250_100recs.pkl"
    results_filename = "DLS_Project/Test20_w2vNCF_mfdim256_with_dot.txt"

    BATCHSIZE = BATCHSIZE * mirrored_strategy.num_replicas_in_sync

    df_train = pd.read_csv("DLS_Project/" + train_filename + ".csv")

    class_weights = class_weight.compute_class_weight(
                   'balanced',
                    np.unique(df_train.interaction.tolist()), 
                    df_train.interaction.tolist()) 

    train_user_n, train_product_n = df_train.user_id.nunique(), 49688
    # print (df_train.shape)

    print (train_filename)

    ds_train, ds_val = make_tf_dataset_train(df_train, ['interaction'], batch_size=BATCHSIZE)
    del df_train
    gc.collect()
    df_train = pd.DataFrame()

    with mirrored_strategy.scope():

        print ("batchsize:", BATCHSIZE)
        # policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
        # tf.keras.mixed_precision.experimental.set_policy(policy) 

        ncf_model = create_ncf(architecture, dropout_input, dropout_hidden_layer, dropout_on_multiplication_boolean, dropout_on_multiplication, batch_norm_boolean, train_user_n, train_product_n,
                               latent_dim_mf, latent_dim_mlp, reg_mf, reg_mlp, dense_layers, reg_layers)

        ncf_model.compile(
            optimizer=Adam(learning_rate=0.001),
    #         loss="binary_crossentropy",
            loss='binary_crossentropy',
            #loss="hinge",
            metrics=[
                tf.keras.metrics.TruePositives(name="tp"),
                tf.keras.metrics.FalsePositives(name="fp"),
                tf.keras.metrics.TrueNegatives(name="tn"),
                tf.keras.metrics.FalseNegatives(name="fn"),
                tf.keras.metrics.BinaryAccuracy(name="accuracy"),
                tf.keras.metrics.Precision(name="precision"),
                tf.keras.metrics.Recall(name="recall"),
                tf.keras.metrics.AUC(name="auc")
            ],
            weighted_metrics=[tf.keras.metrics.BinaryAccuracy(name="weighted_accuracy"),
                tf.keras.metrics.Precision(name="weighted_precision"),
                tf.keras.metrics.Recall(name="weighted_recall"),
                tf.keras.metrics.AUC(name="weighted_auc")]
        )
        ncf_model._name = "neural_collaborative_filtering"
        print (ncf_model.summary())
        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch=(1,2))
        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_precision", patience=2, restore_best_weights = True,
        )

        lr_scheduler_callback = keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)

        train_hist = ncf_model.fit(
            ds_train,
            validation_data=ds_val,
            epochs=N_EPOCHS,
            #callbacks=[tensorboard_callback, early_stopping_callback],
            #callbacks=[tensorboard_callback, lr_scheduler_callback, MemoryPrintingCallback()],
            #callbacks=[tensorboard_callback],
            callbacks=[lr_scheduler_callback],
            verbose=1,
            shuffle=True,
            workers=12,
            use_multiprocessing=True,
            max_queue_size=500,
        )

    df_val, df_truth = load_val_make_preds(ncf_model, val_filename, ground_truth_filename)

    map_15, ndcg_15 = evaluate_and_save_results(df_val, df_truth)

    print (map_15, ndcg_15)
    
    
    # Comment if you don't want to write out results

    file1 = open(results_filename,"a")
    L = [str(train_filename) + "_" + str(architecture) + "_" + str(dropout_input) + "_" + str(dropout_hidden_layer) +
                                "_" + str(dropout_on_multiplication_boolean) + "_" + str(dropout_on_multiplication) + "_" + str(latent_dim_mlp) + "_" + str(latent_dim_mf) +
                                "_" + str(reg_mf) + "_" + str(reg_mlp) + "_" + str(dense_layers) + "_" + str(reg_layers) + "_" + str(N_EPOCHS) + "_" + str(BATCHSIZE) + "_" + str(batch_norm_boolean) + "_" + str(dropout_on_multiplication_boolean) + 
                                                      "_" + str(dropout_on_multiplication) + "\n",
        "MAP 15: " + str(map_15) + "\n",
        "NDCG 15: " + str(ndcg_15) + "\n", "\n \n"]
    file1.writelines(L)
    file1.close() #to change file access modes
    
    save_model_artifacts(train_hist, ncf_model, train_filename, architecture, dropout_input, dropout_hidden_layer, dropout_on_multiplication_boolean, dropout_on_multiplication,
                            latent_dim_mf, latent_dim_mlp, reg_mf, reg_mlp, dense_layers, reg_layers, N_EPOCHS, BATCHSIZE)

    del ncf_model 
    tf.keras.backend.clear_session()
    gc.collect()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



interaction_prior_20k_rank250_100recs
batchsize: 256
Model: "neural_collaborative_filtering"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
product_id (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
mlp_user_embedding (Embedding)  (None, 180)          3711600     user_id[0][0]                    
__________________________________________________________________________________________________
mlp_item_embedding (Embedding)  (None, 180)          8943840     product_id[0][0]                 
________________

/opt/conda/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


0.20357142605012496 0.337229498480553


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

0.1881840104657408 0.3220408590430409


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



interaction_prior_20k_rank250_100recs
batchsize: 256
Model: "neural_collaborative_filtering"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
product_id (InputLayer)         [(None,)]            0                                            
__________________________________________________________________________________________________
mlp_user_embedding (Embedding)  (None, 180)          3711600     user_id[0][0]                    
__________________________________________________________________________________________________
mlp_item_embedding (Embedding)  (None, 180)          8943840     product_id[0][0]                 
________________

/opt/conda/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


0.1897171915647746 0.32364431276545963


# Pipeline 2 End

In [ ]:
# train_hist.history['loss']

# train_hist.history['weighted_precision']

# train_hist.history['val_weighted_precision']

# type(train_hist)

## TO DO

1. Test out if more epochs produces better results given 6 layers on MLP side
2. Test to see if higher dropout rate affects more epochs with 6 layers
3. Test to see if more mlp regularization affects more epochs with 6 layers
4. Test to see if smaller step size affects more epochs with 6 layers
5. Test to see if best hyperparameter set also performs well on the rank 425 set as well as the 300 rec, 100 rec, 150 rec sets
6. Test to see how a lower batch size affects performance on MAP@15

### Saving and Loading Model

In [ ]:
# Saving Model

pd.DataFrame.from_dict(
    train_hist.history).to_csv('DLS_Project/train_history_10epochs_withhn.csv')

ncf_model.save('DLS_Project/10epochs_nohardnegatives_batchsize32_withhn')

print ('done')

In [ ]:
pd.read_csv("DLS_Project/train_history_10epochs_withhn.csv")

pd.read_csv("DLS_Project/train_history_10epochs_withhn.csv")['val_weighted_precision']

### Loading Model and Validation Set & Making Predictions

In [ ]:
#ncf_model = keras.models.load_model('CG_NCF_40k_Test5_9Layers_VanillaNCF_256/')

### Load Validation Set

df_val = pd.read_csv("DLS_Project/interaction_train_20k_rank250_100recs.csv")
df_truth = pd.read_pickle("DLS_Project/cg_interaction_ground_truth_20k_rank250_100recs.pkl")

# all_products = products.product_id.unique()
# all_products.sort()
# product_zerobased_map = dict(zip(all_products, range(len(all_products))))
# df_val.product_id = df_val.product_id.map(product_zerobased_map)

ds_test, _ = make_tf_dataset_val(df_val, ['interaction'], val_split=0, seed=None)
print ('done making batch dataset')

#%%time
tic = time.perf_counter()
ncf_predictions = ncf_model.predict(ds_test, batch_size=512, max_queue_size=500, verbose=0)
toc = time.perf_counter()
print ('done making predictions')
print (f"in {toc - tic:0.4f}")

df_val["ncf_predictions"] = ncf_predictions
df_val["pred"] = (df_val.ncf_predictions >= 0.5) * 1.0
df_val["correct"] = (df_val.interaction == df_val.pred) * 1.0
df_val.drop_duplicates(inplace=True)

### Eye Test

In [ ]:
user_study = 5388

In [ ]:
# df_truth[['user_id']].drop_duplicates().sort_values('user_id').iloc[0:20]
# df_val[['user_id']].drop_duplicates().sort_values('user_id').iloc[0:20]
products = pd.read_csv("products.csv")
products.product_id = products.product_id - 1
df_val_complete = pd.merge(df_val, products[['product_id', 'product_name']], how='left', on='product_id')
df_val_complete[(df_val_complete.user_id == user_study)].sort_values("ncf_predictions", ascending=False).iloc[0:15]

In [ ]:
df_val_complete[(df_val_complete.user_id == user_study) & (df_val_complete.interaction == 1)]

In [ ]:
# df_truth = pd.merge(df_truth, products[['product_id', 'product_name']], how='left', on='product_id')
# df_truth[df_truth.user_id == 4550]

In [ ]:
# df_truth.to_csv("allusers_rank350_groundtruth.csv")

# df_val_complete[(df_val_complete.user_id == 22140)]

# df_val_complete.to_csv("allusers_rank350_preds.csv")

In [ ]:
SparkContext.setSystemProperty('spark.executor.memory', '32g')
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .config("spark.driver.memory", "64g")\
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)
sc.setLogLevel("ERROR")

std = df_val.ncf_predictions.std()
if std < 0.01:
    raise ValueError("Model predictions have standard deviation of less than 1e-2.")

all_indices_predictions = df_val.user_id.unique()
all_indices_predictions.sort()

sorted_df_val = df_val[['user_id', 'product_id', 'ncf_predictions']].sort_values(by=['user_id', 'ncf_predictions'], ascending=[True, False])
gb_sorted_df_val = sorted_df_val.groupby('user_id')
gb_df_truth = df_truth.groupby('user_id')

predictionAndLabelsAll = []
counter = 0
for i in all_indices_predictions:
    if counter % 5000 == 0:
        print (counter)
    counter+=1 
    preds = gb_sorted_df_val.get_group(i).product_id.tolist()
    truth = gb_df_truth.get_group(i).product_id.tolist()
    predictionAndLabelsAll.append((preds, truth))

predictionAndLabels_All = sc.parallelize(predictionAndLabelsAll)
metrics_all = RankingMetrics(predictionAndLabels_All)
print (metrics_all.meanAveragePrecisionAt(15))
print (metrics_all.ndcgAt(15))
print (metrics_all.precisionAt(15))

In [ ]:
# 0.0633
# 0.1486

(.2078-0.1798)/(0.1798)

In [ ]:
print (metrics_all.meanAveragePrecisionAt(100))
print (metrics_all.ndcgAt(100))
print (metrics_all.precisionAt(100))
print (metrics_all.recallAt(100))

In [ ]:
# 1 epoch --> .2132, .3471

In [ ]:
df_truth.user_id.nunique()

In [ ]:
df_val.user_id.nunique()

In [ ]:
user_id_list = all_indices_predictions

df_results = pd.DataFrame(user_id_list, columns=['user_id'])

file_name = 'user_study_dict_ALLUSERS.pkl'

open_file = open(file_name, "rb")
dict_user_id = pickle.load(open_file)
open_file.close()

reverse_user_map = {v: k for k, v in dict_user_id.items()}

df_results["MAP_15"] = None
df_results["NDCG_15"] = None

df_results["true_user_id"] = df_results.user_id.map(reverse_user_map)

for i in range(len(predictionAndLabelsAll)):
    if i%10==0:
        print (i)
    prediction_and_labels = sc.parallelize(predictionAndLabelsAll[i:i+1])
    metrics_all = RankingMetrics(prediction_and_labels)
    df_results.loc[i, "MAP_15"] = metrics_all.meanAveragePrecisionAt(15)
    df_results.loc[i, "NDCG_15"] = metrics_all.ndcgAt(15)
    
df_results.to_pickle("df_results_ALLUSERS_v1.pkl")

In [ ]:
# df_val = pd.merge(df_val, products, how='left', on='product_id')

# df_val.head()



# df_train = pd.merge(df_train, products, how='left', on='product_id')

# df_val.head()

### Past Purchases

# df_train[(df_train.user_id == 1557) & (df_train.interaction == 1)].product_name.value_counts().iloc[0:15]

# df_train[(df_train.user_id == 1557) & (df_train.interaction == 0)].product_name.value_counts().iloc[0:15]

# df_val[(df_val.user_id == 1557) & (df_val.interaction == 0)].product_name.value_counts().iloc[0:15]

### Actual Purchases

# df_val[(df_val.user_id == 1557) & (df_val.interaction == 1)].product_name.value_counts().iloc[0:15]

# df_val.user_id.unique()[0:10]

# df_val[(df_val.user_id == 2402)].sort_values("ncf_predictions", ascending=False).iloc[0:15]

# df_val[(df_val.user_id == 2402) & (df_val.interaction == 1)].sort_values("ncf_predictions", ascending=False)

In [ ]:
# std = df_val.ncf_predictions.std()
# if std < 0.01:
#     raise ValueError("Model predictions have standard deviation of less than 1e-2.")


# df_ncf_predictions = pd.DataFrame()
# df_test = pd.DataFrame()


# def split(a, n):
#     k, m = divmod(len(a), n)
#     return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

# counter = 0
# for index_chunk in list(split(df_val.user_id.unique(), 10)):
#     counter += 1
#     if counter%2==0:
#         print (counter)
#     df_ncf_predictions = df_ncf_predictions.append(df_val[df_val.user_id.isin(index_chunk)].pivot(
#     index="user_id", columns="product_id", values="ncf_predictions"))
    
#     df_test = df_test.append(df_truth[df_truth.user_id.isin(index_chunk)].pivot(
#         index="user_id", columns="product_id", values="interaction"))
    
    
# #     df_test = df_test.append(df_val[df_val.user_id.isin(index_chunk)].pivot(
# #     index="user_id", columns="product_id", values="interaction"))

# data = {}

# data["ncf_predictions"] = df_ncf_predictions.values
# data["test"] = df_test.values

# # data["test"] = df_test.values

# # data = {}

# # data["ncf_predictions"] = df_val.pivot(
# #     index="user_id", columns="product_id", values="ncf_predictions"
# # ).values

# # data["test"] = df_val.pivot(
# #     index="user_id", columns="product_id", values="interaction"
# # ).values

# print ('done')


# # capturing all the users in df_ncf_predictions and sorting them

# all_indices_predictions = df_ncf_predictions.index.tolist()

# all_indices_predictions.sort()
# all_indices_predictions
# print (len(all_indices_predictions))

# # capturing all the products in df_ncf_predictions and df_test

# all_products_predictions = df_ncf_predictions.columns.to_list()
# all_products_results = df_test.columns.to_list()

# prediction_dict = dict(zip(range(len(all_products_predictions)), all_products_predictions))
# result_dict = dict(zip(range(len(all_products_results)), all_products_results))

# SparkContext.setSystemProperty('spark.executor.memory', '32g')
# spark = SparkSession.builder \
#     .master("local[1]") \
#     .appName("SparkByExamples.com") \
#     .config("spark.driver.memory", "64g")\
#     .getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")
# sc = SparkContext.getOrCreate() 
# sqlContext = SQLContext(sc)
# sc.setLogLevel("ERROR")


# predictionAndLabelsAll = []
# counter = 0
# for i in all_indices_predictions:
#     if counter % 10 == 0:
#         print (counter)
#     counter+=1
#     ncf_predictions_tuple = list(enumerate(df_ncf_predictions.loc[i]))
#     ncf_predictions_tuple = [x for x in list(enumerate(df_ncf_predictions.loc[i])) if not np.isnan(x[1])]
#     preds_complete = [x for x in Sort_Tuple(ncf_predictions_tuple)[0:100]]
#     preds = [x[0] for x in Sort_Tuple(ncf_predictions_tuple)[0:100]]
# #     print (preds_complete)
# #     print ('--------------------------------------------------------------------------------------------------------------')
#     preds = [prediction_dict[x] for x in preds]
# #     print (preds)
# #     print ('--------------------------------------------------------------------------------------------------------------')
#     test_tuple = list(enumerate(df_test.loc[i]))
#     test_tuple = [x for x in list(enumerate(df_test.loc[i])) if not np.isnan(x[1])]
#     truth = [x[0] for x in Sort_Tuple(test_tuple) if x[1] == 1]
#     truth = [result_dict[x] for x in truth]
#     predictionAndLabelsAll.append((preds, truth))
# #     print (truth)
#     break
# predictionAndLabels_All = sc.parallelize(predictionAndLabelsAll)
# metrics_all = RankingMetrics(predictionAndLabels_All)
# print (metrics_all.meanAveragePrecisionAt(15))
# print (metrics_all.ndcgAt(15))
# print (metrics_all.precisionAt(15))


In [ ]:
user_id_list = all_indices_predictions

df_results = pd.DataFrame(user_id_list, columns=['user_id'])

file_name = 'user_zerobased_map_for_als_comparison.pkl'

open_file = open(file_name, "rb")
dict_user_id = pickle.load(open_file)
open_file.close()

reverse_user_map = {v: k for k, v in dict_user_id.items()}

df_results["MAP_15"] = None
df_results["NDCG_15"] = None

df_results["true_user_id"] = df_results.user_id.map(reverse_user_map)

In [ ]:
df_results[df_results.user_id == 1557]

In [ ]:
df_user_history = pd.read_pickle('user_history.pkl')

#df_results = df_results.drop('user_id', axis=1)

df_results = pd.merge(df_results, df_user_history, how='left', left_on='true_user_id', right_on='user_id')

In [ ]:
# df_results.sort_values("MAP_15", ascending=False).iloc[0:20]

In [ ]:
# df_val[df_val.user_id == 68142]

In [ ]:
#all_indices_predictions

In [ ]:
for i in range(len(predictionAndLabelsAll)):
    if i%10==0:
        print (i)
    prediction_and_labels = sc.parallelize(predictionAndLabelsAll[i:i+1])
    metrics_all = RankingMetrics(prediction_and_labels)
    df_results.loc[i, "MAP_15"] = metrics_all.meanAveragePrecisionAt(15)
    df_results.loc[i, "NDCG_15"] = metrics_all.ndcgAt(15)

In [ ]:
df_results.to_pickle("df_results_Jan6.pkl")

In [ ]:
indices = list(range(data['test'].shape[0]))
split_n = 4

# precision_scores_all = []
# recall_scores_all = []
predictionAndLabelsAll = []

for index_chunk in list(split(indices, split_n)):
    predictionAndLabels = []

    counter = 0
    precision_scores = []
    for i in index_chunk:
        print (i)
        if i % 10 == 0:
            print (i)
        
#         m_ndcg = tf.keras.metrics.NDCGMetric(topn=15)
#         m_ndcg.update_state(data["test"][i], data["ncf_predictions"][i])
#         ndcg_scores_all.append(m_ndcg.result().numpy())
#         m_ndcg.reset_states()
        
#         print (counter)
#         counter += 1
        ncf_predictions_tuple = list(enumerate(data['ncf_predictions'][i]))
        ncf_predictions_tuple = [x for x in list(enumerate(data['ncf_predictions'][i])) if not np.isnan(x[1])]
#         print (list(enumerate(data['ncf_predictions'][i])))
        #test_tuple = list(enumerate(data['test'][i]))
        #test_tuple = [x for x in list(enumerate(data['test'][i])) if not np.isnan(x[1])]
        preds = [x[0] for x in Sort_Tuple(ncf_predictions_tuple)[0:100]]
        #truth = [x[0] for x in Sort_Tuple(test_tuple) if x[1] == 1]
        print (preds)
        print (truth)
        break
#         predictionAndLabels.append((preds, truth))
        predictionAndLabelsAll.append((preds, truth))
#         predictionAndLabels_ind.append((preds, truth))
#         metrics = RankingMetrics(predictionAndLabels_ind)
#         print (metrics.meanAveragePrecisionAt(15))
#     predictionAndLabels = sc.parallelize(predictionAndLabels)
#     metrics = RankingMetrics(predictionAndLabels)
#     print (metrics.precisionAt(15))
#     print (metrics.meanAveragePrecisionAt(15))
    break
predictionAndLabels_All = sc.parallelize(predictionAndLabelsAll)
metrics_all = RankingMetrics(predictionAndLabels_All)
print (metrics_all.precisionAt(15))
print (metrics_all.meanAveragePrecisionAt(15))
print (metrics_all.ndcgAt(15))

In [ ]:
df_ncf_predictions

In [ ]:
df_ncf_predictions

In [ ]:
# df_val[(df_val.user_id == 10301) & (df_val.product_id == 49234)]

In [ ]:
# import seaborn as sns
# sns.distplot(df_val.ncf_predictions)

In [ ]:
# y_test = df_val.interaction.tolist()
# prob = df_val.ncf_predictions.tolist()

In [ ]:
# # check if model is well calibrated 

# from sklearn.calibration import calibration_curve
# import matplotlib.pyplot as plt

# x, y = calibration_curve(y_test, prob, n_bins = 10, normalize = True)

# # Plot perfectly calibrated
# plt.plot([0, 1], [0, 1], linestyle = '--', label = 'Ideally Calibrated')
 
# # Plot model's calibration curve
# plt.plot(y, x, marker = '.', label = 'Support Vector Classifier')
 
# leg = plt.legend(loc = 'upper left')
# plt.xlabel('Average Predicted Probability in each bin')
# plt.ylabel('Ratio of positives')
# plt.show()

In [ ]:
del df_val
gc.collect()
df_val = pd.DataFrame()

In [ ]:
del ds_test
ds_test = None

In [ ]:
del _

In [ ]:
# def get_model_memory_usage(batch_size, model):
#     import numpy as np
#     try:
#         from keras import backend as K
#     except:
#         from tensorflow.keras import backend as K

#     shapes_mem_count = 0
#     internal_model_mem_count = 0
#     for l in model.layers:
#         layer_type = l.__class__.__name__
#         if layer_type == 'Model':
#             internal_model_mem_count += get_model_memory_usage(batch_size, l)
#         single_layer_mem = 1
#         out_shape = l.output_shape
#         if type(out_shape) is list:
#             out_shape = out_shape[0]
#         for s in out_shape:
#             if s is None:
#                 continue
#             single_layer_mem *= s
#         shapes_mem_count += single_layer_mem

#     trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
#     non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

#     number_size = 4.0
#     if K.floatx() == 'float16':
#         number_size = 2.0
#     if K.floatx() == 'float64':
#         number_size = 8.0

#     total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
#     gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
#     return gbytes

# def keras_model_memory_usage_in_bytes(model, *, batch_size: int):
#     """
#     Return the estimated memory usage of a given Keras model in bytes.
#     This includes the model weights and layers, but excludes the dataset.

#     The model shapes are multipled by the batch size, but the weights are not.

#     Args:
#         model: A Keras model.
#         batch_size: The batch size you intend to run the model with. If you
#             have already specified the batch size in the model itself, then
#             pass `1` as the argument here.
#     Returns:
#         An estimate of the Keras model's memory usage in bytes.

#     """
#     default_dtype = tf.keras.backend.floatx()
#     shapes_mem_count = 0
#     internal_model_mem_count = 0
#     for layer in model.layers:
#         if isinstance(layer, tf.keras.Model):
#             internal_model_mem_count += keras_model_memory_usage_in_bytes(
#                 layer, batch_size=batch_size
#             )
#         single_layer_mem = tf.as_dtype(layer.dtype or default_dtype).size
#         out_shape = layer.output_shape
#         if isinstance(out_shape, list):
#             out_shape = out_shape[0]
#         for s in out_shape:
#             if s is None:
#                 continue
#             single_layer_mem *= s
#         shapes_mem_count += single_layer_mem

#     trainable_count = sum(
#         [tf.keras.backend.count_params(p) for p in model.trainable_weights]
#     )

#     non_trainable_count = sum(
#         [tf.keras.backend.count_params(p) for p in model.non_trainable_weights]
#     )
#     print (non_trainable_count + trainable_count)
#     total_memory = (
#         batch_size * shapes_mem_count
#         + internal_model_mem_count
#         + trainable_count
#         + non_trainable_count
#     )
#     return total_memory

# get_model_memory_usage(512, ncf_model)

# keras_model_memory_usage_in_bytes(model=ncf_model, batch_size=512)/1000000000

# from model_profiler import model_profiler

# batch_size = 512
# profile = model_profiler(ncf_model, batch_size)
# print (profile)

# Precision at K: Keras (Wrong Evaluation Metric)

In [ ]:
# split_n = 5
# precision_scores = []
# indices = list(range(data['test'].shape[0]))

# import random
# random.shuffle(indices)

# # list(split(indices, 10))

# for index_chunk in list(split(indices, split_n)):
#     #print (index_chunk)
#     print
#     precision_ncf = tf.keras.metrics.Precision(top_k=TOP_K)
#     recall_ncf = tf.keras.metrics.Recall(top_k=TOP_K)
    
# #     map_k = tf.compat.v1.metrics.average_precision_at_k(labels=data["test"][index_chunk], predictions=data["ncf_predictions"][index_chunk], k=TOP_K)
#     #map_k.update_state(data["test"][index_chunk], data["ncf_predictions"][index_chunk])
# #     print(
# #         f"At K = {TOP_K}, we have a MAP of {map_k.result().numpy():.5f}"
# #     )
    
# #     map_k = None

#     precision_ncf.update_state(data["test"][index_chunk], data["ncf_predictions"][index_chunk])
#     #recall_ncf.update_state(data["test"][:80], data["ncf_predictions"][:90])

#     print(
#         f"At K = {TOP_K}, we have a precision of {precision_ncf.result().numpy():.5f}"
#     )
#     precision_scores.append(precision_ncf.result().numpy())
#     #, f"and a recall of {recall_ncf.result().numpy():.5f}"
#     precision_ncf = None
#     recall_ncf = None
#     print ('')

# print (np.mean(precision_scores))

In [ ]:
# Function to sort the list by second item of tuple
def Sort_Tuple(tup): 
  
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    return(sorted(tup, key = lambda x: x[1], reverse=True))

from pyspark.mllib.evaluation import RankingMetrics

from pyspark.sql.functions import broadcast
from pyspark import StorageLevel
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.mllib.evaluation import RankingMetrics

SparkContext.setSystemProperty('spark.executor.memory', '32g')
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .config("spark.driver.memory", "64g")\
    .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled","true")
sc = SparkContext.getOrCreate() 
sqlContext = SQLContext(sc)
sc.setLogLevel("ERROR")

In [ ]:
user_id_list = list(df_ncf_predictions.index)

df_results = pd.DataFrame(user_id_list, columns=['user_id'])

file_name = 'user_zerobased_map_for_als_comparison.pkl'

open_file = open(file_name, "rb")
dict_user_id = pickle.load(open_file)
open_file.close()

reverse_user_map = {v: k for k, v in dict_user_id.items()}

df_results["MAP_15"] = None
df_results["NDCG_15"] = None

df_results["true_user_id"] = df_results.user_id.map(reverse_user_map)

In [ ]:
# Sort_Tuple([x for x in list(enumerate(data['ncf_predictions'][9])) if not np.isnan(x[1])])

In [ ]:
# for x in list(enumerate(data['ncf_predictions'][9])):
#     print (x, x[1], np.isnan(x[1]))
#     break

In [ ]:
df_test

In [ ]:
df_ncf_predictions

In [ ]:
indices = list(range(data['test'].shape[0]))
split_n = 4

precision_scores_all = []
recall_scores_all = []
# ndcg_scores_all = []
predictionAndLabelsAll = []

for index_chunk in list(split(indices, split_n)):
    predictionAndLabels = []

    counter = 0
    precision_scores = []
    for i in index_chunk:
        print (i)
        if i % 10 == 0:
            print (i)
        
#         m_ndcg = tf.keras.metrics.NDCGMetric(topn=15)
#         m_ndcg.update_state(data["test"][i], data["ncf_predictions"][i])
#         ndcg_scores_all.append(m_ndcg.result().numpy())
#         m_ndcg.reset_states()
        
#         print (counter)
#         counter += 1
        ncf_predictions_tuple = list(enumerate(data['ncf_predictions'][i]))
        ncf_predictions_tuple = [x for x in list(enumerate(data['ncf_predictions'][i])) if not np.isnan(x[1])]
#         print (list(enumerate(data['ncf_predictions'][i])))
        #test_tuple = list(enumerate(data['test'][i]))
        #test_tuple = [x for x in list(enumerate(data['test'][i])) if not np.isnan(x[1])]
        preds = [x[0] for x in Sort_Tuple(ncf_predictions_tuple)[0:100]]
        #truth = [x[0] for x in Sort_Tuple(test_tuple) if x[1] == 1]
        print (preds)
        print (truth)
        break
#         predictionAndLabels.append((preds, truth))
        predictionAndLabelsAll.append((preds, truth))
#         predictionAndLabels_ind.append((preds, truth))
#         metrics = RankingMetrics(predictionAndLabels_ind)
#         print (metrics.meanAveragePrecisionAt(15))
#     predictionAndLabels = sc.parallelize(predictionAndLabels)
#     metrics = RankingMetrics(predictionAndLabels)
#     print (metrics.precisionAt(15))
#     print (metrics.meanAveragePrecisionAt(15))
    break
predictionAndLabels_All = sc.parallelize(predictionAndLabelsAll)
metrics_all = RankingMetrics(predictionAndLabels_All)
print (metrics_all.precisionAt(15))
print (metrics_all.meanAveragePrecisionAt(15))
print (metrics_all.ndcgAt(15))

In [ ]:
# precision_scores_all = []
# recall_scores_all = []
# # ndcg_scores_all = []
# predictionAndLabelsAll = []

# for index_chunk in list(split(indices, split_n)):
#     predictionAndLabels = []

#     counter = 0
#     precision_scores = []
#     for i in index_chunk:
#         print (i)
#         m_precision = tf.keras.metrics.Precision(top_k=15)
#         m_precision.update_state(data["test"][i], data["ncf_predictions"][i])
#         precision_scores.append(m_precision.result().numpy())
#         precision_scores_all.append(m_precision.result().numpy())
#         m_precision.reset_states()
        
#         m_recall = tf.keras.metrics.Recall(top_k=15)
#         m_recall.update_state(data["test"][i], data["ncf_predictions"][i])
#         recall_scores_all.append(m_recall.result().numpy())
#         m_recall.reset_states()
        
# #         m_ndcg = tf.keras.metrics.NDCGMetric(topn=15)
# #         m_ndcg.update_state(data["test"][i], data["ncf_predictions"][i])
# #         ndcg_scores_all.append(m_ndcg.result().numpy())
# #         m_ndcg.reset_states()
        
# #         print (counter)
# #         counter += 1
#         ncf_predictions_tuple = list(enumerate(data['ncf_predictions'][i]))
#         test_tuple = list(enumerate(data['test'][i]))
#         preds = [x[0] for x in Sort_Tuple(ncf_predictions_tuple)[0:100]]
#         truth = [x[0] for x in Sort_Tuple(test_tuple) if x[1] == 1]
#         predictionAndLabels.append((preds, truth))
#         predictionAndLabelsAll.append((preds, truth))
#     predictionAndLabels = sc.parallelize(predictionAndLabels)
#     metrics = RankingMetrics(predictionAndLabels)
#     print (metrics.precisionAt(15))
#     print (metrics.meanAveragePrecisionAt(15))
    
# predictionAndLabels_All = sc.parallelize(predictionAndLabelsAll)
# metrics_all = RankingMetrics(predictionAndLabels_All)
# print (metrics_all.precisionAt(15))
# print (metrics_all.meanAveragePrecisionAt(15))

In [ ]:
for i in range(len(predictionAndLabelsAll)):
    prediction_and_labels = sc.parallelize(predictionAndLabelsAll[i:i+1])
    metrics_all = RankingMetrics(prediction_and_labels)
    df_results.loc[i, "MAP_15"] = metrics_all.meanAveragePrecisionAt(15)
    df_results.loc[i, "NDCG_15"] = metrics_all.ndcgAt(15)

In [ ]:
#df_results.to_pickle("df_truth_and_prediciton_NCF.pkl")

In [ ]:
# assert len(precision_scores_all) == df_results.shape[0]
# assert len(recall_scores_all) == df_results.shape[0]


# df_results["MAP_15"] = pd.Series(precision_scores_all)
# df_results["Recall_15"] = pd.Series(recall_scores_all)
# df_results["NDCG_15"] = None

In [ ]:
df_results

In [ ]:
df_results[df_results..user_id == 6974]

In [ ]:
df_user_history = pd.read_pickle('user_history.pkl')

df_results = df_results.drop('user_id', axis=1)

df_results = pd.merge(df_results, df_user_history, how='left', left_on='true_user_id', right_on='user_id')

df_results.to_pickle("df_truth_and_prediciton_MF_Only_Dec28_v12.pkl")

In [ ]:
embeddings = ncf_model.get_layer("mf_user_embedding").get_weights()[0]

In [ ]:
# data['test'].shape

ncf_model.get_layer("mf_user_embedding").get_weights()[0].shape

In [ ]:
item_embeddings = ncf_model.get_layer("mf_item_embedding").get_weights()[0]

In [ ]:
user_embeddings = ncf_model.get_layer("mf_user_embedding").get_weights()[0]

In [ ]:
file_name = "product_embedding_MF.pkl"
open_file = open(file_name, "wb")
pickle.dump(item_embeddings, open_file)
open_file.close()

file_name = "user_embedding_MF.pkl"
open_file = open(file_name, "wb")
pickle.dump(user_embeddings, open_file)
open_file.close()

In [ ]:
# predictionAndLabels = []
# precision_scores = []
# counter = 0
# for i in range(0, 1315):
#     print (counter)
#     counter += 1
#     ncf_predictions_tuple = list(enumerate(data['ncf_predictions'][i]))
#     test_tuple = list(enumerate(data['test'][i]))
#     preds = [x[0] for x in Sort_Tuple(ncf_predictions_tuple)[0:100]]
#     truth = [x[0] for x in Sort_Tuple(test_tuple)[0:15] if x[1] == 1]
#     predictionAndLabels.append((preds, truth))
# predictionAndLabels = sc.parallelize(predictionAndLabels)
# metrics = RankingMetrics(predictionAndLabels)
# print (metrics.precisionAt(15))

In [ ]:
# predictionAndLabels = sc.parallelize(predictionAndLabels)
# metrics = RankingMetrics(predictionAndLabels)
# print (metrics.precisionAt(15))

In [ ]:
ncf_predictions_tuple = list(enumerate(data['ncf_predictions'][1]))
test_tuple = list(enumerate(data['test'][1]))

In [ ]:
ncf_predictions_tuple[0:15]

In [ ]:
data['test'][1].sum()

In [ ]:
Sort_Tuple(test_tuple)[0:15]

In [ ]:
[x[0] for x in Sort_Tuple(test_tuple)[0:15] if x[1] == 1]

In [ ]:
[x[0] for x in Sort_Tuple(ncf_predictions_tuple)[0:15]]

In [ ]:
hey.sort()
list(reversed(hey))[0:15]

In [ ]:
precision_ncf.reset_states()

precision_ncf = tf.keras.metrics.Precision(top_k=TOP_K)
recall_ncf = tf.keras.metrics.Recall(top_k=TOP_K)

precision_ncf.reset_states()
precision_ncf.update_state(data["test"][0:1315], data["ncf_predictions"][0:1315])
#recall_ncf.update_state(data["test"][:80], data["ncf_predictions"][:90])

print(
    f"At K = {TOP_K}, we have a precision of {precision_ncf.result().numpy():.5f}"
)

#, f"and a recall of {recall_ncf.result().numpy():.5f}"

In [ ]:
products = pd.read_csv("~/work/products.csv", usecols=['product_id', 'product_name'])

In [ ]:
# df_val = pd.merge(df_val, products, on='product_id', )

In [ ]:
df_val['product_id_original']  = df_val.product_id.apply(lambda x: zerobased_index_to_product[x])

In [ ]:
df_val = pd.merge(df_val, products, left_on='product_id_original', right_on='product_id', how='left')

In [ ]:
df_val.user_id.unique()

In [ ]:
print ([zerobased_index_to_user[x] for x in df_val.user_id.unique()])

In [ ]:
user_to_study = 16274

In [ ]:
df_val[(df_val.user_id == user_to_zerobased_index[user_to_study]) &
       (df_val.interaction == 1)]

In [ ]:
df_val[(df_val.user_id == user_to_zerobased_index[user_to_study])].sort_values("ncf_predictions", ascending=False).iloc[0:15]